<a href="https://colab.research.google.com/github/muffindonor/cloud_computing_project/blob/main/Updated4_Shay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# התקנת ספריית Firebase Admin
!pip install firebase

# התקנת חבילות נדרשות במידת הצורך
!pip install firebase beautifulsoup4

In [48]:
import requests
from bs4 import BeautifulSoup
import re
from collections import defaultdict
from urllib.parse import urljoin
from firebase import firebase
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords


class SearchEngine:
    def __init__(self):
        """Initialize the search engine"""
        self.pages = []
        self.word_locations = defaultdict(list)  # word -> [(page_id, frequency), ...]
        self.stop_words = set(stopwords.words('english'))
        self.firebase_conn = firebase.FirebaseApplication('https://cloudproject-27420-default-rtdb.firebaseio.com/', None)

    def is_valid_term(self, term):
        """
        Validate if a term should be included in the index based on specified rules.

        Rules:
        1. Term must be longer than 2 characters
        2. Term must not contain numbers
        3. Term must only contain English letters
        4. Term must not contain underscores or special characters
        """
        # Check if term is too short (rule 1)
        if len(term) <= 2:
            return False

        # Check if term contains only English letters (rules 2, 3, and 4)
        if not re.match(r'^[a-zA-Z]+$', term):
            return False

        return True

    def fetch_pages(self, base_url, num_pages=5):
        """Fetch pages from the provided base URL"""
        try:
            response = requests.get(base_url)
            soup = BeautifulSoup(response.text, 'html.parser')

            links = []
            for a_tag in soup.find_all('a', href=True):
                full_url = urljoin(base_url, a_tag['href'])
                if full_url.startswith(base_url):
                    links.append(full_url)

            links = list(set(links))[:num_pages]
            print(f"Found {len(links)} links. Processing the first {num_pages}.")

            for link in links:
                try:
                    page_response = requests.get(link)
                    page_soup = BeautifulSoup(page_response.text, 'html.parser')

                    # Extract text from all visible elements
                    for script in page_soup(["script", "style", "meta", "link"]):
                        script.decompose()

                    page_text = page_soup.get_text(" ", strip=True)

                    self.pages.append({
                        'id': link,
                        'title': page_soup.title.string if page_soup.title else 'No Title',
                        'url': link,
                        'content': page_text
                    })
                    print(f"Retrieved: {link}")
                except Exception as e:
                    print(f"Error processing {link}: {str(e)}")
                    continue

            return True

        except Exception as e:
            print(f"Error fetching pages: {str(e)}")
            return False

    def merge_index_data(self, existing_data, new_data):
        """Merge existing index data with new data"""
        merged = defaultdict(list)

        # Convert existing data to dictionary for easier lookup
        if existing_data:
            for entry in existing_data:
                # Only merge terms that pass validation
                if self.is_valid_term(entry['Term']):
                    merged[entry['Term']] = entry['DocIDs']

        # Merge new data
        for term, locations in new_data.items():
            # Only merge terms that pass validation
            if self.is_valid_term(term):
                existing_locations = merged[term]
                # Check for duplicate DocIDs
                existing_doc_ids = {loc['DocID'] for loc in existing_locations}

                # Add only new locations
                for new_loc in locations:
                    if new_loc['DocID'] not in existing_doc_ids:
                        existing_locations.append(new_loc)
                        existing_doc_ids.add(new_loc['DocID'])

                merged[term] = existing_locations

        return [{'Term': term, 'DocIDs': locations} for term, locations in merged.items()]

    def build_index(self):
        """Build or update the word location index"""
        # Clear local index before building new one
        self.word_locations.clear()

        for page in self.pages:
            content = page['content']
            word_counts = defaultdict(int)

            # Process content for each word
            for word in re.findall(r'\b\w+\b', content):
                word_lower = word.lower()
                if word_lower not in self.stop_words and self.is_valid_term(word_lower):
                    word_counts[word_lower] += 1

            # Process compound words
            for compound in re.finditer(r'\b[A-Z][a-zA-Z]*[A-Z][a-zA-Z]*\b', content):
                word = compound.group()
                parts = re.findall(r'[A-Z][a-z]*', word)
                for part in parts:
                    part_lower = part.lower()
                    if part_lower not in self.stop_words and self.is_valid_term(part_lower):
                        word_counts[part_lower] += 1

            # Add counts to index
            for word, count in word_counts.items():
                self.word_locations[word].append({
                    'DocID': page['id'],
                    'Title': page['title'],
                    'Frequency': count
                })

        print("Local index built successfully!")

        existing_index = self.firebase_conn.get('/', 'word_index')
        new_index_data = {term: locations for term, locations in self.word_locations.items()}
        merged_index = self.merge_index_data(existing_index, new_index_data)
        self.firebase_conn.put('/', 'word_index', merged_index)
        print("Index updated in Firebase successfully!")

    def search(self, query, num_results=5):
        """Search for a query in the indexed data"""
        query_words = [word.lower() for word in re.findall(r'\w+', query)
                      if word.lower() not in self.stop_words and self.is_valid_term(word.lower())]
        if not query_words:
            return []

        page_scores = defaultdict(lambda: {'matches': 0, 'total_freq': 0})
        # Get matches for both exact query and compound words containing query
        for entry in self.word_locations.get(query, []):
            page_id = entry['DocID']
            page_scores[page_id]['matches'] += 1
            page_scores[page_id]['total_freq'] += entry['Frequency']

        ranked_results = sorted(
            [(page_id, scores['matches'], scores['total_freq']) for page_id, scores in page_scores.items()],
            key=lambda x: (x[1], x[2]), reverse=True
        )

        results = []
        for page_id, matches, total_freq in ranked_results[:num_results]:
            page = next((p for p in self.pages if p['id'] == page_id), None)
            if page:
                results.append({
                    'title': page['title'],
                    'url': page['url'],
                    'matching_words': matches,
                    'total_frequency': total_freq
                })
        return results

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
# Logic for search and result screen
from firebase import firebase

class SearchLogic:
    def __init__(self, firebase_url):
        self.firebase_conn = firebase.FirebaseApplication(firebase_url, None)
        self.index_data = self.fetch_index()

    # def fetch_index(self):
    #     try:
    #         index = self.firebase_conn.get('/word_index', None)
    #         return index if index else []
    #     except Exception as e:
    #         print(f"Error fetching index from Firebase: {e}")
    #         return []
    def fetch_index(self):
          try:
              index = self.firebase_conn.get('/word_index', None)
              if isinstance(index, dict):
                  # Convert dictionary to list of entries if needed
                  index = [{'Term': term, **data} for term, data in index.items()]
              return index if index else []
          except Exception as e:
              print(f"Error fetching index from Firebase: {e}")
              return []

    def search_query(self, query):
        query_words = [word.lower() for word in query.split()]
        results = []
        for term in query_words:
            for entry in self.index_data:
                if entry['Term'] == term:
                    for occurrence in entry['DocIDs']:
                        results.append({
                            'page_id': occurrence['DocID'],
                            'title': occurrence['Title'],
                            'frequency': occurrence['Frequency']
                        })
        return results

    def update_index(self, term, updated_data):
        try:
            self.firebase_conn.put('/word_index', term, updated_data)
            self.index_data = self.fetch_index()  # Refresh the index data
            return True
        except Exception as e:
            print(f"Error updating index in Firebase: {e}")
            return False


In [50]:
# Style for search and result screen
import ipywidgets as widgets
from IPython.display import display, clear_output

class SearchUI:
    def __init__(self, logic):
        self.logic = logic
        self.current_index = None

    def show_search_screen(self):
        clear_output()

        background_style = '''
        <style>
            body {
                background-color: #1a202c !important;
                min-height: 100vh;
                margin: 0;
                padding: 20px;
                box-sizing: border-box;
            }
            .widget-button button {
                background: linear-gradient(90deg, #ef4444 0%, #f59e0b 100%) !important;
                border: none !important;
                color: white !important;
                border-radius: 20px !important;
                padding: 8px 25px !important;
                font-weight: bold !important;
                transition: transform 0.2s !important;
                box-shadow: none !important;
            }
            .widget-button button:hover {
                transform: scale(1.05) !important;
            }
            .widget-text input {
                background-color: #2d3748 !important;
                border: none !important;
                color: white !important;
                border-radius: 9999px !important;
                padding: 15px 25px !important;
                width: 100% !important;
            }
            .widget-text input::placeholder {
                color: #718096 !important;
            }
            .admin-button {
                background-color: #3b82f6 !important;
                color: white !important;
                padding: 8px 16px !important;
                border-radius: 6px !important;
                border: none !important;
                cursor: pointer !important;
                position: absolute !important;
                right: 20px !important;
                top: 20px !important;
            }
        </style>
        '''

        logo_html = '''
        <div style="text-align: center; margin: 40px auto;">
            <div style="width: 150px; height: 150px; margin: 0 auto; position: relative;">
                <div style="
                    width: 150px;
                    height: 150px;
                    border-radius: 50%;
                    background: white;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    position: relative;
                    z-index: 1;
                ">
                    <span style="color: #718096; font-size: 16px;">150 × 150</span>
                </div>
                <div style="
                    position: absolute;
                    top: -4px;
                    left: -4px;
                    right: -4px;
                    bottom: -4px;
                    background: linear-gradient(90deg, #ef4444 0%, #f59e0b 50%, #ef4444 100%);
                    border-radius: 50%;
                    z-index: 0;
                "></div>
            </div>
        </div>
        '''

        # Admin button
        admin_button = widgets.Button(
            description="Admin Page",
            button_style="warning",
            layout=widgets.Layout(width="auto")
        )

        # Search bar and button
        search_bar = widgets.Text(
            placeholder="Search...",
            layout=widgets.Layout(width="400px")
        )

        search_button = widgets.Button(
            description="Badger Search",
            button_style="warning",
            layout=widgets.Layout(width="auto")
        )

        # Define handlers
        def on_admin_clicked(b):
            self.show_admin_page()

        def on_search_clicked(b):
            query = search_bar.value.strip()
            if query:
                results = self.logic.search_query(query)
                self.show_results_screen(query, results)
            else:
                output = widgets.HTML("<div style='color: #ef4444; text-align: center; margin-top: 10px;'>Please enter a valid query!</div>")
                display(output)

        # Attach handlers
        admin_button.on_click(on_admin_clicked)
        search_button.on_click(on_search_clicked)

        # Display elements
        display(widgets.HTML(background_style))
        display(admin_button)
        display(widgets.HTML(logo_html))

        search_container = widgets.VBox([search_bar, search_button],
            layout=widgets.Layout(
                display='flex',
                flex_flow='column',
                align_items='center',
                width='100%',
                max_width='700px',
                margin='0 auto'
            )
        )
        display(search_container)

    def show_admin_page(self):
        clear_output()

        background_style = '''
        <style>
            body {
                background-color: #1a202c !important;
                min-height: 100vh;
                margin: 0;
                padding: 20px;
                box-sizing: border-box;
            }
            .widget-button button {
                background: linear-gradient(90deg, #ef4444 0%, #f59e0b 100%) !important;
                border: none !important;
                color: white !important;
                border-radius: 20px !important;
                padding: 8px 25px !important;
                font-weight: bold !important;
            }
            .dropdown select {
                background-color: #2d3748 !important;
                color: white !important;
                padding: 10px !important;
                border-radius: 10px !important;
                width: 100% !important;
                margin-bottom: 20px !important;
            }
        </style>
        '''

        header = widgets.HTML('<h2 style="color: white; text-align: center;">Admin Dashboard</h2>')

         # Dropdown for index selection
        dropdown = widgets.Dropdown(
            options=[(str(idx['Term']), idx) for idx in self.logic.index_data],
            description='Select Index:',
            layout=widgets.Layout(width='400px')
        )

        # Text area for DocIDs
        text_area = widgets.Textarea(
            placeholder='DocIDs will appear here when an index is selected...',
            layout=widgets.Layout(width='600px', height='300px')
        )

        # Update button (initially disabled)
        update_button = widgets.Button(
            description="Update DocIDs",
            button_style="warning",
            layout=widgets.Layout(width="150px"),
            disabled=True
        )

        back_button = widgets.Button(
            description="Back to Search",
            button_style="warning",
            layout=widgets.Layout(width="150px")
        )

        # Status message
        status = widgets.HTML('')

        def format_docids(docids):
            return '\n'.join([f"{doc['DocID']}: {doc['Frequency']}" for doc in docids])

        # def parse_docids(text):
        #     docids = []
        #     for line in text.strip().split('\n'):
        #         if line.strip():
        #             parts = line.split(':')
        #             if len(parts) == 2:
        #                 docids.append({'DocID': parts[0].strip(), 'Frequency': int(parts[1].strip())})
        #     return docids
        def parse_docids(text):
            docids = []
            for line in text.strip().split('\n'):
                if line.strip():
                    try:
                        # Split on the last occurrence of ':' to handle URLs with colons
                        docid, frequency = line.rsplit(':', 1)
                        docids.append({
                            'DocID': docid.strip(),
                            'Frequency': int(frequency.strip()),
                            'Title': docid.strip()  # Using DocID as Title
                        })
                    except ValueError:
                        raise ValueError(f"Invalid format in line: {line}")
            return docids

        def on_dropdown_change(change):
            if change['new']:
                self.current_index = change['new']
                docids = self.current_index.get('DocIDs', [])
                text_area.value = format_docids(docids)
                update_button.disabled = False

        def on_text_change(change):
            if self.current_index:
                update_button.disabled = False

        # def on_update_clicked(b):
        #     try:
        #         new_docids = parse_docids(text_area.value)
        #         self.current_index['DocIDs'] = new_docids
        #         status.value = '<div style="color: green;">DocIDs updated successfully!</div>'
        #     except Exception as e:
        #         status.value = f'<div style="color: red;">Error: {str(e)}</div>'
        def on_update_clicked(b):
            try:
                # Parse the text area content
                new_docids = parse_docids(text_area.value)

                if not new_docids:
                    status.value = '<div style="color: #EF4444;">Error: Cannot update with empty DocIDs</div>'
                    return

                # Update the Firebase database
                self.current_index['DocIDs'] = new_docids
                try:
                    # Update the specific term in Firebase
                    self.logic.firebase_conn.put(
                        '/word_index',
                        self.current_index['Term'],
                        self.current_index
                    )

                    # Refresh the index data
                    self.logic.index_data = self.logic.fetch_index()

                    # Update status and disable button
                    status.value = '<div style="color: #10B981;">✓ DocIDs updated successfully!</div>'
                    update_button.disabled = True

                    # Debug output
                    print(f"Updated DocIDs for term '{self.current_index['Term']}': {new_docids}")

                except Exception as e:
                    status.value = f'<div style="color: #EF4444;">Firebase update error: {str(e)}</div>'
                    print(f"Firebase update error: {str(e)}")

            except ValueError as e:
                status.value = f'<div style="color: #EF4444;">Error: {str(e)}</div>'


        def on_back_clicked(b):
            self.show_search_screen()

        # Attach handlers
        dropdown.observe(on_dropdown_change, names='value')
        text_area.observe(on_text_change, names='value')
        update_button.on_click(on_update_clicked)
        back_button.on_click(on_back_clicked)

        # Display elements
        display(widgets.HTML(background_style))
        display(widgets.VBox([
            header,
            dropdown,
            text_area,
            widgets.HBox([update_button, back_button], layout=widgets.Layout(justify_content='center')),
            status
        ], layout=widgets.Layout(align_items='center')))



    def show_results_screen(self, query, results):
      clear_output()

      display(widgets.HTML('''
      <style>
          body {
              background-color: #1a202c !important;
              color: white !important;
          }
          .widget-button button {
              background: linear-gradient(90deg, #ef4444 0%, #f59e0b 100%) !important;
              border: none !important;
              color: white !important;
              border-radius: 20px !important;
              padding: 8px 25px !important;
              font-weight: bold !important;
          }
      </style>
      '''))

      term_count = sum([r['frequency'] for r in results])
      doc_count = len(results)

      header = widgets.HTML(
          f'''
          <div style="margin: 20px; color: white;">
              <h3>Results for: <i>{query}</i></h3>
              <p>Your search appeared <b>{term_count}</b> times in the following links<br>
              About <b>{doc_count}</b> results</p>
          </div>
          '''
      )

      results_html = []
      for result in results:
          results_html.append(f'''
              <div style="display: flex; justify-content: space-between;
              align-items: center; margin: 15px 0; padding: 10px; color: white;">
                  <a href="{result['page_id']}"
                  style="color: #3b82f6; font-size: 16px; text-decoration: none;">
                      {result['title']}
                  </a>
                  <span style="color: #718096;">Frequency: {result['frequency']}</span>
              </div>
          ''')

      results_widget = widgets.HTML('\n'.join(results_html))

      back_button = widgets.Button(
          description="Back to Search",
          button_style="warning",
          layout=widgets.Layout(width="150px")
      )

      def on_back_clicked(b):
          self.show_search_screen()

      back_button.on_click(on_back_clicked)
      display(widgets.VBox([header, results_widget, back_button]))



In [51]:
# Logic init
firebase_url = "https://cloudproject-27420-default-rtdb.firebaseio.com/"
logic = SearchLogic(firebase_url)

# Style init
ui = SearchUI(logic)
ui.show_search_screen()

HTML(value='\n      <style>\n          body {\n              background-color: #1a202c !important;\n          …